The method described below identifies whethear a dataset is of high value.
A dataset is considered of high value when the bbox it defines aplies to a whole country.
In the code below we consider datasets that contain only one bbox. 
We have defined a threshold if the bounding box covers at least 70% of the country to consider a dataset
of high value. This threshold can be changed based on our needs

In [1]:
import xml.etree.ElementTree as ET
from shapely.geometry import Point, Polygon, box
import geopandas as gpd
from geopy import distance
from pyproj import CRS
import pandas as pd
import folium

Load the "ne_110m_admin_0_countries" dataset which is part of the Natural Earth collection.<br/>
This will be used for conducting geographical analysis and more specifically in identification of the reference country that is being processed. <br/>
Futhermore we load the eu_countries.csv which contains the Country Name, the Country Code and the Area.

In [2]:
# Load world dataset from local shapefile
world = gpd.read_file("./world_countries/ne_110m_admin_0_countries.shp")

# Load counties details
eu_countries_df = pd.read_csv("./eu_countries.csv")

world = world[world['NAME'].isin(eu_countries_df["Country Name"])]

Load the coordinates from xml

In [3]:
root = ET.parse("./dataset.xml")

ns = {'gmd': 'http://www.isotc211.org/2005/gmd',
      'gco': 'http://www.isotc211.org/2005/gco'}

bbox = root.find('.//gmd:EX_GeographicBoundingBox', ns)

coords = None

if bbox is not None:
    west = float(bbox.find('./gmd:westBoundLongitude/gco:Decimal', ns).text)
    east = float(bbox.find('./gmd:eastBoundLongitude/gco:Decimal', ns).text)
    south = float(bbox.find('./gmd:southBoundLatitude/gco:Decimal', ns).text)
    north = float(bbox.find('./gmd:northBoundLatitude/gco:Decimal', ns).text)
    coords = (west, south, east, north)

print(f"Coordinates: {coords}")

Coordinates: (6.54747184, 51.2249633, 11.74161086, 53.93431815)


Plot the bounding box using the folium library for visualization

In [4]:
if coords:
    # Calculate the center of the bounding box
    center_lat = (coords[1] + coords[3]) / 2
    center_lon = (coords[0] + coords[2]) / 2

    # Create a map centered on the bounding box
    m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

    # Add the bounding box rectangle to the map
    folium.Rectangle(
        bounds=[(coords[1], coords[0]), (coords[3], coords[2])],
        color="red",
        fill=True,
        fillColor="red",
        fillOpacity=0.2
    ).add_to(m)

    # Display the map
    display(m)
else:
    print("No valid coordinates found in the XML file.")

The identify_country function finds the country corresponding to a bounding box by:
<ul>
<li>Checking which countries intersect the box.
<li>If any intersect, it sees if the box's centroid is inside a country.
<li>If not, it reprojects to an equal-area CRS and selects the country with the largest overlap. It returns the country's name or None if no match is found.
</ul>

In [5]:
def identify_country(west, south, east, north, world):
    try:
        bbox = box(west, south, east, north)
        bbox_gdf = gpd.GeoDataFrame({'geometry': [bbox]}, crs=world.crs)

        # Find intersecting countries
        intersecting = world[world.intersects(bbox)]
        
        if not intersecting.empty:
            # Calculate centroid of the bbox
            bbox_centroid = Point((west + east) / 2, (south + north) / 2)

            # print("bbox centroid",bbox_centroid)
            # Find countries that contain the centroid
            containing = intersecting[intersecting.contains(bbox_centroid)]

            if not containing.empty:
                return containing.iloc[0]['NAME']
            else:
                # If no country contains the centroid, return the one with the largest overlap
                # Project to Equal Area projection for accurate area calculation
                equal_area_crs = CRS.from_epsg(6933)
                # Transform countries that overlap to equal-area-map
                intersecting_projected = intersecting.to_crs(equal_area_crs)
                # Transform bbox on equal_area_crs
                bbox_projected = bbox_gdf.to_crs(equal_area_crs)
                
                intersections = intersecting_projected.intersection(bbox_projected.iloc[0].geometry)
                max_intersection_idx = intersections.area.idxmax()
                return intersecting.loc[max_intersection_idx, 'NAME']
        
        return None
    except Exception as e:
        print(f"Error in identify_country: {e}")
        return "Error occurred"

The get_area function calculates the area of the bbox. <br/>
It projects the coordinates from epsg:4326 to EPSG:3035 to get accurate area measurement since this
is EU specific projection

In [6]:
def get_area(coords):
    west, south, east, north = coords
    polygon = [(west, south), (east, south), (east, north), (west, north)]
    polygon_geom = Polygon(polygon)
    # This creates a GeoDataFrame setting the Global Coordinate System
    poly_df = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])
    # Project to european coordinate system since datasets explored are tailored to EU
    poly_df = poly_df.to_crs('EPSG:3035')
    # Divide by 1.000.000 to get it in square kilometers
    return round(poly_df.area[0] / 1000000,3)

In [7]:
identified_country = identify_country(west, south, east, north, world)
bounding_box_area = get_area(coords)
country_area = eu_countries_df.loc[eu_countries_df['Country Name'] == identified_country, 'Area'].values[0]

if identified_country:
    bounding_box_area = get_area(coords)
    country_area = eu_countries_df.loc[eu_countries_df['Country Name'] == identified_country, 'Area'].values[0]

    # Calculate the ratio of bounding box area to country area
    area_ratio = bounding_box_area / country_area

    # Define a threshold for what we consider a "national" dataset
    national_threshold = 0.7  # e.g., if the bounding box covers at least 70% of the country

    is_national = area_ratio >= national_threshold

    print(f"Identified Country: {identified_country}")
    print(f"Bounding Box Area: {bounding_box_area:.2f} km²")
    print(f"Country Area: {country_area:.2f} km²")
    print(f"Area Ratio: {area_ratio:.2f}")
    print(f"High Value Dataset: {is_national}")


Identified Country: Germany
Bounding Box Area: 106045.57 km²
Country Area: 357763.10 km²
Area Ratio: 0.30
High Value Dataset: False
